# DAML 10 - Neural Networks

Michal Grochmal <michal.grochmal@city.ac.uk>

If we place several perceptrons (neurons) together
we can slice the search space with several lines.
This is called an *Artificial Neural Network* (ANN)
It is not trivial to train these neurons,
after all we do not know what the output of most neurons should be.

But let's just use `sklearn`'s `MLPClassifier` (multi-layer-perceptron-classifier)
as a model and see if we can make it work.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-talk')

## Glass Dataset

The forensics of glass composition can reveal the provenience of a piece of glass,
yet different brands of glass are slightly different from each other.
We will build a model which will classify glass based on its composition,
and will use online learning so that we are ready to learn from new data at any time.

This is a dataset at UCI machine learning repository, we need to build our `load_glass` function.

In [ ]:
import os
import sys
import requests
import numpy as np
from sklearn import datasets
from sklearn.utils import Bunch


def load_glass():
    glass_dir = 'uci_glass'
    data_dir = datasets.get_data_home()
    data_path = os.path.join(data_dir, glass_dir, 'glass.data')
    descr_path = os.path.join(data_dir, glass_dir, 'glass.names')
    glass_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/glass/glass.data'
    glass_descr = 'https://archive.ics.uci.edu/ml/machine-learning-databases/glass/glass.names'
    os.makedirs(os.path.join(data_dir, glass_dir), exist_ok=True)
    try:
        with open(descr_path, 'r') as f:
            descr = f.read()
    except IOError:
        print('Downloading file from', glass_descr, file=sys.stderr)
        r = requests.get(glass_descr)
        with open(descr_path, 'w') as f:
            f.write(r.text)
        descr = r.text
        r.close()
    try:
        data = np.loadtxt(data_path, delimiter=',')
    except IOError:
        print('Downloading file from', glass_data, file=sys.stderr)
        r = requests.get(glass_data)
        with open(data_path, 'w') as f:
            f.write(r.text)
        r.close()
        data = np.loadtxt(data_path, delimiter=',')
    target = data[:, 10].astype(np.int).copy()
    target[target > 3] -= 1  # fix non-existent classes
    return Bunch(DESCR=descr,
                 data=data[:, :10].copy(),
                 feature_names=['ID', 'RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe'],
                 target=target,
                 target_names=['windows_float_processed',
                               'windows_non_float_processed',
                               'vehicle_windows',
                               'containers',
                               'tableware',
                               'headlamps'])


glass = load_glass()
print(glass.DESCR)

The support of the classes is quite different.
This isn't an easy problem to model.

In [ ]:
pd.Series(glass.target).value_counts()

Without knowing much about this model,
let's try to use a neural network to classify this dataset.
We know that the neural network is a good amount
of interconnected perceptrons and that we perturb the perceptron weights
based on errors in classification to achieve convergence.

We also know that `sklearn` provides us with a simple neural network class,
no harm in trying it out.

In [ ]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(glass.data, glass.target, test_size=0.2, random_state=42)

This is a real dataset,
we may as well try to do things properly and take a test set out.

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

net = MLPClassifier(activation='relu', hidden_layer_sizes=(20,), alpha=0.01, tol=0.001,
                    max_iter=200, solver='sgd', learning_rate='constant')
param_dict = {
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0],
    'tol': [0.001, 0.01, 0.1],
}
grid = GridSearchCV(net, param_dict, cv=9)
grid.fit(xtrain, ytrain)
grid.best_estimator_

Some models may not converge but the cross-validation should root them out.
And the best model should be a converged network.

In [ ]:
grid.best_score_

The score isn't bad but a classification report may give a better view
of how a multilabel classifier performs.

In [ ]:
from sklearn.metrics import classification_report

yfit = grid.best_estimator_.predict(xtest)
print(classification_report(ytest, yfit, target_names=glass.target_names))

And to see the misclassification we use the confusion matrix.

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

m = confusion_matrix(ytest, yfit)
fig = plt.figure(figsize=(12, 12))
ax = sns.heatmap(m.T, square=True, annot=True, fmt='d', cmap='Blues',
                 xticklabels=glass.target_names, yticklabels=glass.target_names)
ax.set(xlabel='true label', ylabel='predicted label');

From the above we learned one of the most important details about neural networks:
since weights are updated based on classification errors between expected and predicted
classes, if we have classes with very little support the network will fail at classifying them.

As we get collect more data the effect of classes with smaller support diminishes.
Neural Nets work in similar fashion to us, humans, you cannot show a person 2 paintings
by Titian and then 200 painting not by Titian and expect her to be an expert at identifying
Titian's work.
ANNs work best the more data you have and the more diverse the data is.

For the glass data we'll leave as an exercise trying out a Random Forest,
that model is very good at digging out classes with small support
(contrary to decision trees!).
But for now let's understand more about ANNs.

## Neural Network Concepts

OK, so how does that work?
We managed to train perceptrons because we knew what the output of each perceptron should be.
And we saw that a perceptron can only output either -1 or 1 with the `sign` function.

In neural networks the perceptrons are organized in layers:

- One *input layer* which has one perceptron per feature
- At least one *hidden layer* with fully connected perceptrons
- One output layer with one perceptron per output class (or just one for regression)

Each perceptron will have its own weights, and these will be trained simultaneously
across the entire network, by a process called *backpropagation*.
But before we dive into backpropagation let's discuss activation functions.
The `sign` function is only one viable activation function,
we can make our ANN work with several others:

In [ ]:
fig, ax = plt.subplots(5, 2, figsize=(16, 14), sharey=True, sharex=True)
ax_size = [-5, 5, -1.5, 3]
fig.subplots_adjust(hspace=0.3)
# sign
x_full = np.linspace(-5, 5, 60)
x1 = np.linspace(-5, 0, 30)
x2 = np.linspace(0, 5, 30)
y1 = x1*0 - 1
y2 = x2*0 + 1
ax[0, 0].plot(np.hstack([x1, x2]), np.hstack([y1, y2]), color='crimson')
ax[0, 0].legend(['$sign()$'])
ax[0, 0].axis(ax_size)
ax[0, 0].set_title('Sign')
# hard limit
y1 = x1*0
y2 = x2*0 + 1
ax[1, 0].plot(np.hstack([x1, x2]), np.hstack([y1, y2]), color='crimson')
ax[1, 0].legend(['$limit()$'])
ax[1, 0].axis(ax_size)
ax[1, 0].set_title('Hard Limit')
# linear
ax[0, 1].plot(x_full, x_full/3, color='crimson')
ax[0, 1].legend(['$y = ax + b$'])
ax[0, 1].axis(ax_size)
ax[0, 1].set_title('Linear')
# relu
y1 = x1*0
y2 = x2
ax[1, 1].plot(np.hstack([x1, x2]), np.hstack([y1, y2/3]), color='crimson')
ax[1, 1].legend(['$y = 0 | y = ax$'])
ax[1, 1].axis(ax_size)
ax[1, 1].set_title('Rectified Linear Unit (ReLU)')
# tanh sigmoid
y_full = (np.exp(x_full) - np.exp(-x_full))/(np.exp(x_full) + np.exp(-x_full))
ax[2, 0].plot(x_full, y_full, color='crimson')
ax[2, 0].legend(['$y = (e^{x} - e^{-x})/(e^{x} + e^{-x})$'])
ax[2, 0].axis(ax_size)
ax[2, 0].set_title('Hyperbolic Tangent (tanh)')
# log sigmoid
y_full = 1/(1 + np.exp(-x_full))
ax[2, 1].plot(x_full, y_full, color='crimson')
ax[2, 1].legend(['$y = 1/(1 + e^{-x})$'])
ax[2, 1].axis(ax_size)
ax[2, 1].set_title('Log-Sigmoid (Logit)')
# radial basis
y_full = np.exp(- x_full*x_full)
ax[3, 0].plot(x_full, y_full, color='crimson')
ax[3, 0].legend(['$y = e^{-x^2}$'])
ax[3, 0].axis(ax_size)
ax[3, 0].set_title('Radial Basis (RBF)')
# triangular basis
x1 = np.linspace(-5, -1, 30)
x2 = np.linspace(-1, 0, 10)
x3 = np.linspace(0, 1, 10)
x4 = np.linspace(1, 5, 30)
y1 = x1*0
y2 = x2 + 1
y3 = 1 - x3
y4 = x4*0
ax[3, 1].plot(np.hstack([x1, x2, x3, x4]), np.hstack([y1, y2, y3, y4]), color='crimson')
ax[3, 1].legend(['$y = ax | y = -ax$'])
ax[3, 1].axis(ax_size)
ax[3, 1].set_title('Triangular Basis')
# satlin sign
x1 = np.linspace(-5, -1, 30)
x2 = np.linspace(-1, 0, 10)
x3 = np.linspace(0, 1, 10)
x4 = np.linspace(1, 5, 30)
y1 = x1*0 - 1
y2 = x2
y3 = x3
y4 = x4*0 + 1
ax[4, 0].plot(np.hstack([x1, x2, x3, x4]), np.hstack([y1, y2, y3, y4]), color='crimson')
ax[4, 0].legend(['$y = -1 | y = 1 | y = ax$'])
ax[4, 0].axis(ax_size)
ax[4, 0].set_title('Satlin Sign')
# satlin
x1 = np.linspace(-5, -1, 30)
x2 = np.linspace(-1, 0, 10)
x3 = np.linspace(0, 1, 10)
x4 = np.linspace(1, 5, 30)
y1 = x1*0
y2 = x2*0
y3 = x3
y4 = x4*0 + 1
ax[4, 1].plot(np.hstack([x1, x2, x3, x4]), np.hstack([y1, y2, y3, y4]), color='crimson')
ax[4, 1].legend(['$y = 0 | y = 1 | y = ax$'])
ax[4, 1].axis(ax_size)
ax[4, 1].set_title('Satlin');

And that's not all, for the last layer of the network two other functions are often used.
The `compet` function, which outputs a unit value for the maximum input and zeros for
all other inputs.

$$\text{compet} = floor \left( \frac{x_i}{max(x)} \right), \text{for i in }1, 2, \dots, N$$

And the `softmax` functions which scales all inputs so that the sum of all inputs equals a unit.

$$\text{softmax} = \frac{e^{x_i}}{\sum_{k = 0}^{N} e^{x_k}} , \text{for i in }1, 2, \dots, N$$



<div style="text-align:right"><sup>Image available as `*ann.svg`</sup></div>

## Backpropagation

How can we train the perceptrons in the hidden layer?
We do not really know what their output should be.
We can certainly say that we have all weights in a vector, let's call is $w$,
and then we can attempt Gradient Descent to find the best matrix.
But GD requires us to have some form of cost (error) function since it is defined as:

$$
\nabla E = \frac{\partial E}{\partial p_1}\hat{\imath} + \frac{\partial E}{\partial p_2}\hat{\jmath} + \ldots
$$

The cost function will be multidimensional, exactly one dimension per weight
in the vector $w$ and the last dimension being the value of the error itself.
The total error of the network will be a sum of the error contribution of each weight
(let's say that the vector $e$ is the amount of error each weight introduces):

$$E = \sum a(w^T \cdot e)$$

Where $a$ is the activation function.

If we know the contribution of error from each weight, we can derivate the function
above and we have all the components of the error vector to perform gradient descent.

Simplifying the problem:
If we know the classification error of one output perceptron and we know its weights,
we can divide it across the weights coming into the perceptron, and then multiply
by the weights to get each weight contribution.
The contribution of each weight coming into the output perceptron is the contribution
of the output of the perceptron in the previous layer.
Summing up the output contributions in the previous layer
we know the error contributions on the output side of each perceprtron on that layer.
We can then repeat the procedure for this layer,
and also get the contributions of the output of the previous layer.
And so on.

In the end we have the error contribution of every weight, which, when applied the
derivate of the activation function, is one of the components of the Gradient
vector of the cost function at the current location.
And going in the opposite direction of the gradient vector in each of its components
(i.e. in the direction of each weight, ignoring the dimension of the error itself)
we are minimizing the error of the function, and therefore making the network classify better!

## Faces

The previous more-or-less real problem we faced was face identification with the LFW dataset.
This is a quite complex dataset but has a good number of instances.
A neural network has a chance of performing a good-enough classification.

In [ ]:
from sklearn.datasets import fetch_lfw_people

faces = fetch_lfw_people(min_faces_per_person=50)
faces.DESCR

To save computations - and time - we will perform PCA just once, on the full set.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=200)
faces_pca = pca.fit_transform(faces.data)
xtrain, xtest, ytrain, ytest = train_test_split(faces_pca, faces.target, test_size=0.2)

The hyperparameters are tuned on the training set only.
This is the most important concept in ML, therefore I'll keep repeating it.

In [ ]:
net = MLPClassifier(hidden_layer_sizes=(300,), alpha=0.001, tol=0.01,
                    max_iter=100, solver='sgd', learning_rate='constant')
param_dict = {
    'alpha': [0.001, 0.01, 0.1, 1.0],
    'tol': [0.001, 0.01],
}
grid = GridSearchCV(net, param_dict, cv=5)
grid.fit(xtrain, ytrain)
grid.best_estimator_

This is a reasonable network, with 300 neurons in the hidden layer.

In [ ]:
grid.best_score_

And the score is reasonable.
But the full report is what we should look at.

In [ ]:
yfit = grid.predict(xtest)
print(classification_report(ytest, yfit, target_names=faces.target_names))

And the confusion matrix to understand misclassification.

In [ ]:
m = confusion_matrix(ytest, yfit)
fig = plt.figure(figsize=(14, 14))
ax = sns.heatmap(m.T, square=True, annot=True, fmt='d', cmap='BuGn',
                 xticklabels=faces.target_names, yticklabels=faces.target_names)
ax.set(xlabel='true label', ylabel='predicted label');

The ANN performs reasonably and we did not even attempt to tune half
of all its hyperparameters.
That said, the number of hyperparameters is at the same time a strength
and a weakness of ANNs.
SGD training uses a lot of randomness, and some hyperparameter tuning
may react badly to randomness.

Let's try to train networks of different sizes,
i.e. using the number of neurons in the hidden layer as a tunable hyperparameter.
We will train each size of the network twice.

In [ ]:
sizes = [10, 30, 60, 100, 200, 300, 600, 900]
scores1 = []
scores2 = []
for size in sizes:
    for s in [scores1, scores2]:
        print('Trianing net of size', size)
        net = MLPClassifier(hidden_layer_sizes=(size,),
                            alpha=0.001, max_iter=100, solver='sgd')
        net.fit(xtrain, ytrain)
        score = net.score(xtest, ytest)
        s.append(score)

And let's have a look how the scores of all these different networks look.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 7))
ax.plot(sizes, scores1)
ax.plot(sizes, scores2);

There is a good deal of randomness in ANN training but we can say that around 300
neurons (perceptrons) in the hidden layer we reached some form of plateau in accuracy score.
The number of neurons in a layer can be used as a hyperparameter alright,
and should be tuned with a grid search.
Different layers in the network may be tuned in size independently,
the computational effort to train a network can be very big.

## References

[UCI - Glass Identification Dataset][1]

[1]: https://archive.ics.uci.edu/ml/datasets/glass+identification "UCI Glass"